## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-19-06-45-56 +0000,nyt,A Reddit Post Led to a Breakthrough in the Bro...,https://www.nytimes.com/2025/12/19/us/brown-mi...
1,2025-12-19-06-34-43 +0000,nypost,Trump reveals ‘Patriot Games’ for high school ...,https://nypost.com/2025/12/19/sports/trump-rev...
2,2025-12-19-06-34-17 +0000,nyt,"Europe to Lend $105 Billion to Ukraine, Withou...",https://www.nytimes.com/2025/12/18/world/europ...
3,2025-12-19-06-25-35 +0000,nyt,What We Know About the Suspect in the Brown an...,https://www.nytimes.com/2025/12/19/us/neves-va...
4,2025-12-19-06-21-51 +0000,bbc,"A 45% school bus fare hike is too much, says dad",https://www.bbc.com/news/articles/cj6906z3drxo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2369/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
7,trump,55
60,new,28
55,bondi,20
21,billion,17
366,year,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
123,2025-12-18-23-26-52 +0000,nypost,GOP senators push Trump to use $5 billion in R...,https://nypost.com/2025/12/18/world-news/gop-s...,118
184,2025-12-18-21-08-55 +0000,nyt,"Trump Media to Merge With TAE Technologies, a ...",https://www.nytimes.com/2025/12/18/business/tr...,118
176,2025-12-18-21-37-00 +0000,wsj,Trump Media to Merge With Nuclear Fusion Firm ...,https://www.wsj.com/business/deals/trump-media...,115
145,2025-12-18-22-28-39 +0000,nypost,President Trump gives federal workers 2 new ho...,https://nypost.com/2025/12/18/us-news/presiden...,109
46,2025-12-19-03-19-00 +0000,wsj,President Trump signed into law new powers to ...,https://www.wsj.com/politics/national-security...,107


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
123,118,2025-12-18-23-26-52 +0000,nypost,GOP senators push Trump to use $5 billion in R...,https://nypost.com/2025/12/18/world-news/gop-s...
336,95,2025-12-18-11-43-19 +0000,nypost,Mourners grieve 10-year-old slain in Bondi mas...,https://nypost.com/2025/12/18/world-news/mourn...
186,76,2025-12-18-21-01-00 +0000,wsj,The Bank of Mexico lowered its benchmark inter...,https://www.wsj.com/articles/bank-of-mexico-ma...
151,51,2025-12-18-22-24-00 +0000,wsj,Economists warned a big drop in November infla...,https://www.wsj.com/economy/the-data-problems-...
142,50,2025-12-18-22-32-00 +0000,wsj,Former Nascar Driver Greg Biffle and Family Ki...,https://www.wsj.com/us-news/small-jet-crashes-...
61,50,2025-12-19-02-25-15 +0000,nypost,"Brown, MIT shooting suspect ID’d as Claudio N...",https://nypost.com/2025/12/18/us-news/brown-un...
78,46,2025-12-19-01-19-33 +0000,bbc,Kennedy Center to be renamed Trump-Kennedy Cen...,https://www.bbc.com/news/articles/cp84pxvp87eo...
257,46,2025-12-18-17-32-16 +0000,bbc,Trump Media to merge with fusion energy firm i...,https://www.bbc.com/news/articles/cp9kv9lld38o...
189,39,2025-12-18-20-58-14 +0000,nypost,"New York Times columnist David Brooks, Google ...",https://nypost.com/2025/12/18/us-news/david-br...
232,38,2025-12-18-18-35-35 +0000,nypost,Israel investigating whether Iran was involved...,https://nypost.com/2025/12/18/world-news/israe...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
